## 確認收斂次數

In [1]:
import unittest
from time import time, ctime

# CONSTANT
DATA_ROOT = "./data"
DBLP_PATH = DATA_ROOT + "/dblp"
AMAZON_PATH = DATA_ROOT + "/amazon"
FACEBOOK_PATH = DATA_ROOT + "/facebook"
NOTIFY_ENDPOINT = r"https://notify.run/O6EfLmG6Tof1s5DljYB7"


import pandas as pd

from package.model import DiffusionModel
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.itemset import ItemsetFlyweight, ItemRelation
from package.utils import getItemsPrice, read_items
from package.algorithm import Algorithm
from package.coupon import Coupon

NUM_TOPICS = 5
TOPICS = {
    "Node": {
        "0": [0.9, 0.1, 0.0],
        "1": [0.2, 0.8, 0.0],
        "2": [0.8, 0.2, 0.0],
        "3": [0.2, 0.4, 0.4],
    },
    "Item": {
        "iPhone": [0.7, 0.0, 0.3],
        "AirPods": [0.9, 0.0, 0.1],
        "Galaxy": [0.0, 0.8, 0.2],
    }
}
PRICES = {
    "iPhone": 50,
    "AirPods": 5,
    "Galaxy": 60,
}
RELATION = pd.DataFrame.from_dict({
            "iPhone":{
                "AirPods":10,
                "Galaxy":-5
            },
            "AirPods":{
                "iPhone":1,
                "Galaxy":0,
            },
            "Galaxy":{
                "iPhone":-8,
                "AirPods":1,
            }
            })

In [3]:
items = read_items(AMAZON_PATH + "/sample_items.csv")

topicModel = TopicModel(NUM_TOPICS)
topicModel.read_topics(node_file=FACEBOOK_PATH + "/nodes_with_" + str(NUM_TOPICS) + "_topic.csv",
                        items_file=AMAZON_PATH + "/items_with_" + str(NUM_TOPICS) + "_topic.csv")

graph = SN_Graph.construct(FACEBOOK_PATH + "/edges", topicModel, located=False)

relation = ItemRelation()
relation.construct(AMAZON_PATH + "/sample_items.csv")
itemset = ItemsetFlyweight(getItemsPrice(AMAZON_PATH + "/sample_items.csv"), topicModel, relation)

model = DiffusionModel("amazon in dblp", graph, itemset, threshold=0)
seed_size = min(itemset.size, graph.number_of_nodes())
seeds = model.selectSeeds(seed_size)
model.allocate(seeds, [itemset[asin] for asin in itemset.PRICE.keys()])

simluation_times = 1
algo = Algorithm(model, 0, simulationTimes=simluation_times)
candidatedCoupons = [Coupon(20, itemset["B000JX5JGI B000VYIUJS"], 5, itemset["B000JX5JGI B000VYIUJS"])]

performanceFile = r"./result/distribution.txt"


for times in range(0,1):
    k=0
    algo.setLimitCoupon(k)
    start_time = time()
    if k == 0:
        outputCoupons, tagger = algo.simulation([])
    else:
        outputCoupons, tagger = algo.simulation(candidatedCoupons)

    end_time = time()

    # with open(performanceFile, "a") as record:
        
    #     record.write("{0},runtime={1},revenue={2},expected_revenue={3},active_node={4},expected_active_node={5},times={6}\n".format(
    #         ctime(end_time),
    #         (end_time - start_time),
    #         tagger["TagRevenue"].amount(),
    #         tagger["TagRevenue"].expected_amount(),
    #         tagger["TagActiveNode"].amount(),
    #         tagger["TagActiveNode"].expected_amount(),
    #         times,
    #         ))
        
    #     for c in outputCoupons:
            
    #         record.write(str(c) + "\n")
    #     record.write("\n")
        

Constructing graph...
Connecting the edges...Done


In [3]:
length, path = SN_Graph.max_product_path(graph, model.getSeeds())


In [6]:
path

{'2543': ['2543'],
 '2347': ['2347'],
 '2407': ['2543', '2407'],
 '2429': ['2543', '2429'],
 '2606': ['2543', '2606'],
 '2247': ['2543', '2247'],
 '2384': ['2543', '2384'],
 '2638': ['2543', '2638'],
 '2150': ['2543', '2150'],
 '2600': ['2543', '2600'],
 '2232': ['2543', '2232'],
 '2506': ['2543', '2506'],
 '2071': ['2543', '2071'],
 '1939': ['2543', '1939'],
 '2369': ['2543', '2369'],
 '2279': ['2543', '2279'],
 '2052': ['2543', '2052'],
 '2340': ['2543', '2340'],
 '2039': ['2543', '2039'],
 '2250': ['2543', '2250'],
 '2111': ['2543', '2111'],
 '2338': ['2543', '2338'],
 '2428': ['2543', '2428'],
 '2196': ['2543', '2196'],
 '2590': ['2543', '2590'],
 '2083': ['2543', '2083'],
 '2241': ['2543', '2241'],
 '2189': ['2543', '2189'],
 '2095': ['2543', '2095'],
 '2463': ['2543', '2463'],
 '2191': ['2543', '2191'],
 '2033': ['2543', '2033'],
 '2471': ['2543', '2471'],
 '1932': ['2543', '1932'],
 '2055': ['2543', '2055'],
 '2478': ['2543', '2478'],
 '2148': ['2543', '2148'],
 '2240': ['2543',

In [4]:
graph.nodes["2347"]

{'desired_set': <package.itemset.Itemset at 0x2c75a185f30>,
 'adopted_set': None,
 'adopted_records': [[<package.itemset.Itemset at 0x2c75a185f30>,
   None,
   12.31]],
 'topic': [0.23875585645156794,
  0.2423318585557119,
  0.18433347348034332,
  0.11359856910512188,
  0.22098024240725514]}

In [13]:
graph.nodes["2543"]

{'desired_set': <package.itemset.Itemset at 0x1254d1b7c40>,
 'adopted_set': None,
 'adopted_records': [[<package.itemset.Itemset at 0x1254d1b7c40>,
   None,
   18.99]],
 'topic': [0.19015006121707884,
  0.10805833054734416,
  0.39984238221800494,
  0.12739171749838238,
  0.1745575085191897]}

In [ ]:
import numpy as np
colors = {"B000VYIUJS":"64e9c0", "B000JX5JGI": "cbb446", "B000JX5JGI B000VYIUJS":"ba9800"}
for k,l in tagger["TagDecidedMainItemset"].items():
    print(k)
    print(l)
    # colors[k] = ((np.random.random(), np.random.random(), np.random.random()))

In [ ]:
import matplotlib.pyplot as plt

plt.figure("Main Itemset")
for k,v in tagger["TagDecidedMainItemset"].items():
    plt.plot(list(range(len(v))), v, 'ko--', color=colors[k], label=k, markerfacecolor='none')
plt.title("Withput coupons")
plt.xlabel("step")
plt.ylabel("times of decied main itemset")
plt.legend()

In [ ]:
str(outputCoupons[0])

### 可累積商品和累積門檻條件的變化對於使用者CP值感知的影響

In [1]:
import pandas as pd

from package.model import DiffusionModel
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.itemset import ItemsetFlyweight, ItemRelation
from package.utils import getItemsPrice, read_items
from package.algorithm import Algorithm
from package.coupon import Coupon
from package.user_proxy import UsersProxy

NUM_TOPICS = 5
TOPICS = {
    "Node": {
        "0": [0.9, 0.1, 0.0],
        "1": [0.2, 0.8, 0.0],
        "2": [0.8, 0.2, 0.0],
        "3": [0.2, 0.4, 0.4],
    },
    "Item": {
        "iPhone": [0.7, 0.0, 0.3],
        "AirPods": [0.9, 0.0, 0.1],
        "Galaxy": [0.0, 0.8, 0.2],
    }
}
PRICES = {
    "iPhone": 260.0,
    "AirPods": 60.0,
    "Galaxy": 500.0,
}
RELATION = pd.DataFrame.from_dict({
            "iPhone":{
                "AirPods":10,
                "Galaxy":-5
            },
            "AirPods":{
                "iPhone":1,
                "Galaxy":0,
            },
            "Galaxy":{
                "iPhone":-8,
                "AirPods":1,
            }
            })

topicModel = TopicModel(3, TOPICS["Node"], TOPICS["Item"])
relation = ItemRelation(RELATION)
itemset = ItemsetFlyweight(prices = PRICES, topic = topicModel.getItemsTopic(), relation=relation)
graph = SN_Graph(topicModel.getNodesTopic())
coupon = Coupon(500, itemset["Galaxy"], itemset["AirPods iPhone"].price*0.6, itemset["AirPods iPhone"])
user_proxy = UsersProxy(graph, itemset, [coupon], 0)
user_id = "0"
main_itemset = itemset["iPhone"]
graph.add_node(user_id, adopted_set=None, desired_set=main_itemset, topic=TOPICS["Node"][user_id])

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

step = 20
# 可累積商品
list_accItemset = [id for id, items in itemset if itemset.issubset(main_itemset, items)]

# 累積門檻
list_accThreshold = list(np.arange(main_itemset.price, itemset["Galaxy iPhone AirPods"].price+step*10, step))

# 折抵商品
list_disItemset = [id for id, items in itemset]

# 折抵金額
list_discount = list(np.arange(0, itemset["Galaxy iPhone AirPods"].price, step=20))

multi_index = []
data = []
results = [] # for debug

for acc_itemset_id in list_accItemset:
    coupon.accItemset = itemset[acc_itemset_id]

    for threshold in list_accThreshold:
        coupon.accThreshold = threshold

        for dis_itemset_id in list_disItemset:
            coupon.disItemset = itemset[dis_itemset_id]

            for discount in list_discount:
                coupon.discount = discount

                user_proxy.setCoupons([coupon])
            
                result = user_proxy.adopt(user_id)
                results.append(result)

                # reset
                graph._initNode(user_id)
                graph.nodes[user_id]["desired_set"] = main_itemset
                
                multi_index.append((acc_itemset_id, threshold, dis_itemset_id, discount))
                data.append([result["VP"], result["amount"], result["tradeOff_items"]])


In [3]:
# create a multiple index dataframe
index = pd.MultiIndex.from_tuples(multi_index, names=('可累積商品', '累積門檻', '折抵商品', '折抵金額'))
df = pd.DataFrame(data=data ,index=index, columns=['VP值', '價格', '購買商品組合'])

In [4]:
pd.set_option('display.max_columns', None)
# df = df.reorder_levels(['可累積商品', '累積門檻', '折抵金額', '折抵商品']).sort_index(level=['可累積商品', '累積門檻', '折抵金額', '折抵商品'])

In [5]:
coupon_test = Coupon(270, itemset["AirPods Galaxy iPhone"], 40, itemset["AirPods"])
graph._initNode(user_id)
print("Main itemset: {0}".format(user_proxy._mainItemsetVP(user_id, main_itemset)))

for id, obj in itemset:
    if itemset.issubset(main_itemset, obj):
        VP = user_proxy._addtionallyAdoptVP(user_id, main_itemset, obj, coupon_test)
        print("{0}: {1:.8f}".format(id, VP))

Main itemset: 0.002423076923076923
iPhone: 0.00233333
AirPods iPhone: 0.00249578
Galaxy iPhone: 0.00044105
AirPods Galaxy iPhone: 0.00064060


In [6]:
"{0:.4E}".format(user_proxy._mainItemsetVP(user_id, main_itemset))

'2.4231E-03'

In [7]:
df = df.reorder_levels(['可累積商品', '累積門檻', '折抵商品', '折抵金額'])

In [8]:
df.loc[(df['價格'] > main_itemset.price) & (df['VP值'] > 0.002423076923076923)]

VP值     價格  \
可累積商品                 累積門檻  折抵商品           折抵金額                    
iPhone                260.0 AirPods        40.0  0.002592  280.0   
                            AirPods Galaxy 40.0  0.002592  280.0   
AirPods iPhone        260.0 AirPods        40.0  0.002592  280.0   
                            AirPods Galaxy 40.0  0.002592  280.0   
Galaxy iPhone         260.0 AirPods        40.0  0.002592  280.0   
                            AirPods Galaxy 40.0  0.002592  280.0   
AirPods Galaxy iPhone 260.0 AirPods        40.0  0.002592  280.0   
                            AirPods Galaxy 40.0  0.002592  280.0   

                                                         購買商品組合  
可累積商品                 累積門檻  折抵商品           折抵金額                  
iPhone                260.0 AirPods        40.0  AirPods iPhone  
                            AirPods Galaxy 40.0  AirPods iPhone  
AirPods iPhone        260.0 AirPods        40.0  AirPods iPhone  
                            AirPods Galaxy 40.0  AirPods iPhone  
Galaxy iPhone         260.0 AirPods        40.0  AirPods iPhone  
                            AirPods Galaxy 40.0  AirPods iPhone  
AirPods Galaxy iPhone 260.0 AirPods        40.0  AirPods iPhone  
                            AirPods Galaxy 40.0  AirPods iPhone

In [10]:
df.loc[("AirPods Galaxy iPhone", 280, "AirPods")]

C:\Users\user\AppData\Local\Temp\ipykernel_19204\1194730405.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[("AirPods Galaxy iPhone", 280, "AirPods")]


,VP值,價格,購買商品組合
折抵金額,,,
0.0,0.002423,260.0,iPhone
20.0,0.002423,260.0,iPhone
40.0,0.002423,260.0,iPhone
60.0,0.002592,260.0,AirPods iPhone
80.0,0.002592,260.0,AirPods iPhone
100.0,0.002592,260.0,AirPods iPhone
120.0,0.002592,260.0,AirPods iPhone
140.0,0.002592,260.0,AirPods iPhone
160.0,0.002592,260.0,AirPods iPhone


1. 將折抵商品分為兩組，有無包含主商品
2. 分成兩個角度觀察
    - 可累積商品為同一組，觀察不同折抵商品組合間，在不同折抵金額下隨著累積門檻越高會有什麼變化
    - 反過來觀察可折抵商品一組

In [ ]:
# 折抵商品不包含主商品
dis_items_exclued_main_items = df[~df.index.get_level_values("折抵商品").str.contains("Galaxy")]
print(dis_items_exclued_main_items)

In [ ]:
# 可累積商品
index = 0
acc_itemset = list_accItemset[index]
select_acc_itemset = dis_items_exclued_main_items[dis_items_exclued_main_items.index.get_level_values("可累積商品") == acc_itemset]
print(list_accItemset)
print(list_accItemset[index])


In [ ]:
colors = ["#242E91", "#91233B", "#23913D", "#ff3399"]
markers = ["o", "v", "s", "p", "x"]

In [ ]:
set(select_acc_itemset.index.get_level_values('折抵商品'))

In [ ]:
pd.set_option('display.max_rows', None)

for distItemset_id in set(select_acc_itemset.index.get_level_values('折抵商品')): # 折抵商品組合
    select_dis_itemset = select_acc_itemset[select_acc_itemset.index.get_level_values("折抵商品") == distItemset_id]
    print(select_dis_itemset)
    # for i in range(len(list_discount)): # 折抵金額
    #     select_discount = select_dis_itemset[select_dis_itemset.index.get_level_values("折抵金額") == list_discount[i]]
    #     print(select_discount)

In [ ]:
for distItemset_id in set(select_acc_itemset.index.get_level_values('折抵商品')): # 折抵商品組合
    select_dis_itemset = select_acc_itemset[select_acc_itemset.index.get_level_values("折抵商品") == distItemset_id]
    print(distItemset_id)
    plt.figure("the relation of VP with discount items: {0}".format(distItemset_id))
    for i in range(len(list_discount)): # 折抵金額
        select_discount = select_dis_itemset[select_dis_itemset.index.get_level_values("折抵金額") == list_discount[i]]
        print(list_discount[i])
        plt.plot(list_accThreshold, list(select_discount["VP值"]), 'k' + markers[i] + '--', 
                color=colors[i], 
                label="{:.0f}% off".format(list_discount[i]*100), 
                markerfacecolor='none')
    plt.title("the relation of VP with discount items: {0}".format(distItemset_id))
    plt.xlabel("Threshold of coupon")
    plt.ylabel("VP")
    plt.legend()

    plt.figure("the relation of amount with discount items: {0}".format(distItemset_id))
    for i in range(len(list_discount)): # 折抵金額
        select_discount = select_dis_itemset[select_dis_itemset.index.get_level_values("折抵金額") == list_discount[i]]
        plt.plot(list_accThreshold, list(select_discount["價格"]), 'k' + markers[i] + '--', 
                color=colors[i], 
                label="{:.0f}% off".format(list_discount[i]*100), 
                markerfacecolor='none')
    plt.title("the relation of amount with discount items: {0}".format(distItemset_id))
    plt.xlabel("Threshold of coupon")
    plt.ylabel("VP")
    plt.legend()